<font size="5">**Installing and importing libraries**</font>

In [1]:
!pip install timm
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 6.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=5c9caff11a68bc7f2f699ac81cd6cf404c7aaf9a5964668977bc789d71b5bc49
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [2]:
import numpy as np
import pandas as pd
import cv2
from matplotlib import pyplot as plt
import torch
import string
from torch.utils.data import Dataset,DataLoader
import timm
import torch
import torch.nn as nn
from tqdm.notebook import tqdm
import os 

In [3]:
info = pd.read_excel("/kaggle/input/images2000/images_info.xlsx")

In [4]:
info.head()

,id,image,caption
0,1,image_1.jpg,A beautiful young girl posing on a white backg...
1,4,image_4.jpg,"One hand pointing, finger out, thumb up"
2,6,image_6.jpg,A beautiful young girl posing with a gray back...
3,3,image_3.jpg,Medical stitches - puncture wound closed up
4,15,image_15.jpg,A happy couple


In [32]:
from sentence_transformers import SentenceTransformer
model_sentence = SentenceTransformer('all-mpnet-base-v2')

<font size="5">**initializing the configration class and datasets**</font>

In [172]:
class CFG:
    model_name = 'resnet101d'
    target_size = 100
    size = 224
    num_workers = 2
    batch = 1
    weight_decay = 1e-2
    train = False

In [173]:
class dataset(Dataset):
    def __init__(self, df,transform= None):
        self.df = df
        self.id = self.df.id
        self.caption = self.df.caption
        self.images = self.df.image
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self,idx):
        self.str = self.images.iloc[idx]
        self.cap = self.caption.iloc[idx]
        embedding = model_sentence.encode(self.cap)
        img_path = f"/kaggle/input/images2000/images/images/{self.str}"
        assert os.path.exists(img_path)
        img = cv2.imread(img_path)
        color = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        color = torch.tensor(color.transpose((2, 0,1)), device = device)/255
        return self.id.iloc[idx], color,embedding

In [174]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

<font size="5">**Model Created**</font>

In [175]:
class CustomNet(nn.Module):
    def __init__(self, model_name=CFG.model_name, pretrained = True):
        super().__init__()
        self.model = timm.create_model(CFG.model_name, pretrained=pretrained)
        self.model.classifier = nn.Linear(1000, CFG.target_size)

    def forward(self, x):
        x = self.model(x)
        x = self.model.classifier(x)
        return x
    

In [176]:
model = CustomNet(model_name=CFG.model_name, pretrained=True)
model = model.to(device)

<font size="5">**Test Function**</font>

In [177]:
def test_fun(test_loader, model, device):
    model.eval()
    preds = []
    ids= []
    embs= []
    for id,images,emb in tqdm(test_loader,total = len(test_loader)):
#         print(images.shape)
        images = images.to(device)
        with torch.no_grad():
            y_preds = model(images)
        preds.append(y_preds.to('cpu').numpy())
        ids.append(id.to('cpu').numpy())
        embs.append(emb.to('cpu').numpy())
    predictions = np.concatenate(preds)
    idx = np.concatenate(ids)
    embs = np.concatenate(embs)
    return  ids,predictions, embs

In [178]:
ds = dataset(info)
dl = DataLoader(ds,batch_size = CFG.batch)

<font size="5">**Features Extraction**</font>

In [179]:
id_ans, y_ans,emb_ans = test_fun(dl,model,device)

  0%|          | 0/2054 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

<font size="5">**Concatenating all features**</font>

In [180]:
sub = pd.DataFrame()
sub['id'] = id_ans
for i in range(len(id_ans)):
    sub['id'][i] = id_ans[i].item()

In [181]:
feat_df = pd.DataFrame(y_ans)
emb_df = pd.DataFrame(emb_ans)
feat_df['id'] = sub['id']
cols = feat_df.columns.tolist()
cols = cols[-1:] + cols[:-1]
feat_df = feat_df[cols]
feat_df = pd.concat((feat_df, emb_df), axis=1)

<font size="5">**Converted to csv format**</font>

In [182]:
feat_df.to_csv("lol.csv", index = False)

In [183]:
feat_df

,id,0,1,2,3,4,5,6,7,8,...,758,759,760,761,762,763,764,765,766,767
0,1,0.876442,0.772697,-0.426554,-0.717973,-0.299369,0.149705,0.504142,0.583282,-0.450843,...,-0.002850,0.096363,0.028717,-0.014735,0.005955,0.096271,-0.017570,0.009647,-0.007925,-0.032737
1,4,-0.068213,-0.801402,0.115911,0.325891,-0.258643,0.284229,-0.090170,0.295825,-0.650482,...,-0.043540,0.002089,0.012997,-0.037303,-0.035503,0.080465,0.010829,-0.025224,-0.006035,-0.034029
2,6,0.732465,0.215382,-0.214630,-0.007758,-0.994771,0.315718,-0.066161,1.400498,0.394435,...,-0.024907,0.089047,0.055288,-0.011224,-0.018265,0.106321,-0.004782,0.019157,-0.019440,-0.017563
3,3,0.034409,-0.166493,-0.304466,0.434057,-0.238237,0.100215,-0.099117,0.540851,-0.430632,...,-0.071692,0.012479,0.036048,-0.049273,0.021836,0.038228,-0.003534,-0.032955,0.000735,0.011938
4,15,0.436154,0.996192,-0.166986,-0.288504,-0.505896,0.358547,0.044804,1.540469,-0.821466,...,-0.042583,-0.023659,-0.071023,-0.013670,-0.010461,0.001962,-0.037601,0.014136,0.064987,0.012270
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2049,2497,0.193200,0.265628,0.272505,0.066187,-0.389322,0.233175,0.086607,0.553969,-0.657289,...,0.037155,0.005176,-0.022121,0.009011,-0.038447,-0.061522,-0.013397,0.044720,0.002051,-0.029341
2050,2499,0.090905,0.033949,-0.361954,0.229257,-0.404773,-0.386816,-0.272679,0.183549,0.145713,...,-0.010756,-0.023584,0.057689,-0.058690,-0.027101,-0.043675,0.023235,-0.027885,-0.026029,0.011551
2051,2498,-0.022966,0.329102,-0.646828,-0.103314,-0.350625,-0.175107,-0.187651,0.119967,-0.554982,...,-0.010151,0.000220,0.019179,-0.046472,-0.007003,0.009775,0.043070,0.001283,-0.026792,0.002127
2052,2500,0.179796,0.035878,0.074447,0.034130,-0.394574,-0.507121,-0.457518,0.607762,0.027087,...,0.016784,0.012092,0.082265,-0.003862,-0.004553,0.054586,0.038444,0.030251,0.012941,0.006542
